Stippling Sandbox

In [1]:
from PIL import Image, ImageDraw, ImageFilter
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np
from rasterize import Scanline_Rasterize_Polygon, Weighted_Raster_Centroid, Raster_BBox
from geometry import Sort_Vertices, Create_Seed

In [2]:
def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)


def Outline_Poly(polygon,color,width):
    for i, point in enumerate(polygon):

        k = i + 1
        if k == len(polygon):
            k = 0
        point2 = polygon[k]

        draw.line((point[0],point[1],point2[0],point2[1]),fill=color, width=width)

Image parameters

In [3]:
# image_resolution = 800
image_path = 'sampling/ring.jpg'
weight_image = Image.open(image_path)
size = 256
weight_image = weight_image.resize((size,size))
width,height = weight_image.size
print(width,height)
image_resolution = width
# weight_image = weight_image.filter(ImageFilter.BoxBlur(2))

# img = Image.open(image_path)
img = weight_image

256 256


In [4]:
def Lloyd_Relax(seeds, image_resolution, image_array):
    vor = Voronoi(seeds)
    boundary_seeds = [
        np.array([-image_resolution, -image_resolution]),
        np.array([2 * image_resolution, -image_resolution]),
        np.array([2 * image_resolution, 2 * image_resolution]),
        np.array([-image_resolution, 2 * image_resolution])
    ]

    polygons = []
    open_cells = []
    for index, region in enumerate(vor.regions):
        poly = []
        if len(region) == 0:
            continue
        elif -1 in region:
            open_cells.append(region)
            continue
        for index in region:
            if index != -1:
                poly.append(vor.vertices[index])
            else:
                print("OUTSIDE")
        
        sorted_poly = Sort_Vertices(poly)

        polygons.append(sorted_poly)

    centroids = []
    for polygon in polygons:

        bbox = Raster_BBox(polygon, image_resolution)
        rastered_polygon = Scanline_Rasterize_Polygon(polygon, bbox, image_resolution)
        centroid = Weighted_Raster_Centroid(rastered_polygon, image_array)
        if centroid:
            if centroid[0] < 0 or centroid[0] >= image_resolution or centroid[1] < 0 or centroid[1] >= image_resolution:
                centroids.append(Create_Seed(image_resolution, 100, weight_image))
            else:
                centroids.append(centroid)
        else:
            centroids.append(Create_Seed(image_resolution, 100, weight_image))
            
    
    new_seeds = boundary_seeds + centroids

    return new_seeds

Creating seeds

In [5]:
seeds = [
    np.array([-image_resolution, -image_resolution]),
    np.array([2 * image_resolution, -image_resolution]),
    np.array([2 * image_resolution, 2 * image_resolution]),
    np.array([-image_resolution, 2 * image_resolution])
]
startseeds = seeds
point_count = 100

## Rejection sampling
threshold = 150
# threshold = 256

n = 0
for i in range(point_count):
    seeds.append(Create_Seed(image_resolution, threshold, weight_image))
# while n < point_count:
    # rand_x = np.random.rand() * image_resolution
    # rand_y = np.random.rand() * image_resolution
    # sampled_value = weight_image.getpixel((rand_x,rand_y))
    # if np.mean(sampled_value) < threshold:
    #     seeds.append(np.array([rand_x,rand_y]))
    #     n += 1

## Seed image
seed_img = Image.open(image_path)
draw = ImageDraw.Draw(seed_img)
for seed in startseeds:
    center_ellipse(seed[0], seed[1],1,'red')

seed_img.save("seed_image.jpg")

In [11]:
image_array = np.array(weight_image)
weight_array = np.zeros([image_resolution,image_resolution])
pixels = 0
for x in range(image_resolution):
    for y in range(image_resolution):
        pixel = image_array[x,y]
        weight_array[x,y] = (np.mean(pixel) / 255)

0


In [7]:
img = Image.new('RGB', (image_resolution * 2, image_resolution * 2))
draw = ImageDraw.Draw(img)
center_rectangle(image_resolution,image_resolution,image_resolution * 2,image_resolution * 2,'white')

for seed in seeds:
    center_ellipse(seed[0] * 2, seed[1] * 2,2,'black')
img.save(f'sequence/stipple_result_{0}.png')

print("starting relaxation")
for i in range(50):
    center_rectangle(image_resolution,image_resolution,image_resolution * 2,image_resolution * 2,'white')
    new_seeds = Lloyd_Relax(seeds,image_resolution, weight_array)

    seeds = new_seeds
    print(f"Iteration {i}")

    # for seed in seeds:
    #     center_ellipse(seed[0] * 2, seed[1] * 2,2,'black')
    # img.save(f'sequence/stipple_result_{i + 1}.png')


starting relaxation
Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36


KeyboardInterrupt: 

In [ ]:
final_scale = 6
img = Image.new('RGB', (image_resolution * final_scale, image_resolution * final_scale))
draw = ImageDraw.Draw(img)
center_rectangle(image_resolution * final_scale / 2,image_resolution * final_scale / 2,image_resolution * final_scale,image_resolution * final_scale,'white')
for seed in seeds:
    center_ellipse(seed[0] * final_scale, seed[1] * final_scale,5,'black')
img.save(f'sequence/stipple_result_scaled.png')